<a href="https://colab.research.google.com/github/s99436q/Wikipedia-and-GPT-2/blob/master/Wiki_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prepare data


In [1]:
#@title set up { form-width: "20%" }
#%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import platform
import time
import pathlib
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/GPT-2')
#print('python version:',platform.python_version())
#print('tensorflow version:',tf.__version__)
#print(f'kears version: {tf.keras.__version__}')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#@title load data { output-height: 2, form-width: "20%" }
dataset, ds_info = tfds.load(
    name='wikipedia/20190301.en',
    data_dir='gs://tfds-data/datasets',
    with_info=True,
    split=tfds.Split.TRAIN,
)
#print(ds_info)
#print(ds_info.splits['train'].num_examples)

#Preprocess the dataset



In [3]:
#@title Flatten the dataset { output-height: 20, form-width: "20%" }
#coverting the datasetfrom the set of articles to set of characters.
def article_to_text(text):
  return np.array([char for char in text.numpy().decode('utf-8')])

dataset_text = dataset.map(
    lambda x: tf.py_function(func=article_to_text, inp=[x['text']], Tout=tf.string))


for x in dataset_text.take(2): 
    print(len(x),x.numpy())

#covert the text dataset to more regular char-based dataset
dataset_chars=dataset_text.unbatch()
for x in dataset_chars.take(2): 
    print(x)#.numpy())#.decode('utf-8'))



17851 [b'J' b'o' b's' ... b'n' b't' b's']
2554 [b'P' b'a' b'u' ... b'e' b'r' b's']
tf.Tensor(b'J', shape=(), dtype=string)
tf.Tensor(b'o', shape=(), dtype=string)


In [4]:
#@title generating vocabulary { output-height: 20, form-width: "20%" }
vocab=set()
for text in dataset_text.take(1000): #ideally we should take all dataset into
    vocab.update([char.decode('utf-8') for char in text.numpy()])
   
vocab=sorted(vocab)
print(f'num of unique vocab:{len(vocab)}\n vocab set:{vocab}')

num of unique vocab:621
 vocab set:['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\xa0', '£', '§', '«', '®', '°', '±', '²', '·', '»', '¼', '½', '¿', 'Á', 'Å', 'Æ', 'Ç', 'É', 'Ë', 'Í', 'Î', 'Ó', 'Ö', '×', 'Ø', 'Ü', 'Þ', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ø', 'ú', 'û', 'ü', 'ý', 'ā', 'ă', 'ą', 'Ć', 'ć', 'Č', 'č', 'đ', 'ė', 'ę', 'ě', 'ğ', 'ġ', 'Ħ', 'ī', 'İ', 'ı', 'ļ', 'Ł', 'ł', 'ń', 'ň', 'Ō', 'ō', 'ő', 'ř', 'Ś', 'ś', 'Ş', 'ş', 'Š', 'š', 'ţ', 'ū', 'ź', 'ż', 'Ž', 'ž', 'ơ', 'ư', 'ǔ',

In [5]:
#@title vectorize the text{form-width: "20%"}
char2index={char: index for index, char in enumerate(vocab)}
index2char=np.array(vocab)

print(index2char)
print(char2index)

['\t' '\n' ' ' '!' '"' '#' '$' '%' '&' "'" '(' ')' '*' '+' ',' '-' '.' '/'
 '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' ':' ';' '<' '=' '>' '?' '@' 'A'
 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S'
 'T' 'U' 'V' 'W' 'X' 'Y' 'Z' '[' ']' '^' '_' '`' 'a' 'b' 'c' 'd' 'e' 'f'
 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x'
 'y' 'z' '{' '|' '}' '~' '\xa0' '£' '§' '«' '®' '°' '±' '²' '·' '»' '¼'
 '½' '¿' 'Á' 'Å' 'Æ' 'Ç' 'É' 'Ë' 'Í' 'Î' 'Ó' 'Ö' '×' 'Ø' 'Ü' 'Þ' 'ß' 'à'
 'á' 'â' 'ã' 'ä' 'å' 'æ' 'ç' 'è' 'é' 'ê' 'ë' 'ì' 'í' 'î' 'ï' 'ñ' 'ò' 'ó'
 'ô' 'õ' 'ö' 'ø' 'ú' 'û' 'ü' 'ý' 'ā' 'ă' 'ą' 'Ć' 'ć' 'Č' 'č' 'đ' 'ė' 'ę'
 'ě' 'ğ' 'ġ' 'Ħ' 'ī' 'İ' 'ı' 'ļ' 'Ł' 'ł' 'ń' 'ň' 'Ō' 'ō' 'ő' 'ř' 'Ś' 'ś'
 'Ş' 'ş' 'Š' 'š' 'ţ' 'ū' 'ź' 'ż' 'Ž' 'ž' 'ơ' 'ư' 'ǔ' 'ș' 'ț' 'ɔ' 'ə' 'ɛ'
 'ʷ' 'ʼ' 'ʿ' '˚' 'Ι' 'Π' 'α' 'β' 'ε' 'η' 'ι' 'κ' 'μ' 'ο' 'ρ' 'ς' 'τ' 'υ'
 'χ' 'ψ' 'ό' 'Б' 'В' 'Д' 'Ж' 'З' 'И' 'К' 'Л' 'М' 'Н' 'О' 'П' 'С' 'У' 'Ф'
 'Х' 'а' 'б' 'в' 'г' 'д' 'е' 'з' 'и' 'й' 'к' 'л' '

In [6]:
#@title transfor text to int{form-width: "20%"}
def char_to_index(char):
    char_symbol=char.numpy().decode('utf-8')
    char_index= char2index[char_symbol] if char_symbol in char2index else char2index['?']
    return char_index
dataset_chars_int= dataset_chars.map(
    lambda char: tf.py_function(func=char_to_index, inp=[char], Tout=tf.int32)
)
for x in dataset_chars_int.take(2):
  print(x.numpy(),index2char[x.numpy()])
print(dataset_chars,dataset_chars_int)

44 J
80 o
<DatasetV1Adapter shapes: <unknown>, types: tf.string> <DatasetV1Adapter shapes: <unknown>, types: tf.int32>


# Prepare Training data

In [7]:
#@title create training sequence{form-width: "20%"}
seq_len =  201#@param {type:"integer"}

dataset_seqs=dataset_chars_int.batch(seq_len+1,drop_remainder=True)
for x in dataset_seqs.take(2):
  print(repr(''.join(index2char[x.numpy()])))
  #print(''.join(index2char[x]))

'Joseph Harold Greenberg (May 28, 1915 – May 7, 2001) was an American linguist, known mainly for his work concerning linguistic typology and the genetic classification of languages.\n\nLife\n\nEarly life and'
' education \n(Main source: Croft 2003)\n\nJoseph Greenberg was born on May 28, 1915 to Jewish parents in Brooklyn, New York. His first great interest was music. At the age of 14, he gave a piano concert in'


In [8]:
#@title make pair sequence{form-width: "20%"}
def split_input_target(chunk):
  input_text=chunk[:-1]
  target_text=chunk[1:]
  return input_text, target_text
dataset_pairs=dataset_seqs.map(split_input_target)
# for inp,out in dataset_pairs.take(1):
#   print(''.join(index2char[inp]))
#   print(''.join(index2char[out]))

#  oseph GreenbergPauline DonaldaList of German football%
#  ---------------------------------------------------------
#  Joseph GreenbergPauline DonaldaList of German football

In [9]:
#@title make batches on sequence pairs {form-width: "20%"}
batch_size=64#@param {type:"integer"}
# Buffer size to shuffle the dataset (TF data is designed to work
# with possibly infinite sequences, so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in
# which it shuffles elements).
buffer_size=100#@param {type:"integer"}
prefetch_size=10 #@param {type:"integer"}
# How many items to prefetch before the next iteration.

dataset_seqs_batches = dataset_pairs\
                      .shuffle(buffer_size)\
                      .batch(batch_size,drop_remainder=True)\
                      .prefetch(prefetch_size)
#ds=dataset_seqs_batches.as_numpy_iterator()                  
#print(next(ds))

# Build Model

In [10]:
#@title build sequential model {form-width: "20%"}
#embedding dimension
#[1,20,1,1] dim=4  Apple -> [1,0,1,1] [...] [...] [...] [1,2,0,1]
vocab_size=len(vocab) 
embedding_dim =  256#@param {type:"integer"}
rnn_units =  1024#@param {type:"integer"}


def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.models.Sequential()
  # Embedding layer
  model.add(tf.keras.layers.Embedding(
      input_dim=vocab_size,
      output_dim=embedding_dim,
      batch_input_shape=[batch_size, None]  
      #         time:0123              embedding layer 
      #batch=3  seq=[I have a apple]       |   [1.2.3.0] [] [] [] [...]
      #         seq=[cat eats apple]       |   [11.1.3.0] [] [] [] [...]
      #         seq=[god hate apple]       |   [0.2.3.1] [] [] [] [...]

  # LSTM layer  //RNN LSTM GRU
  ))
  model.add(tf.keras.layers.LSTM(
      units=rnn_units, #dimensionality of the output space.       
                                       
      return_sequences=True,            #A|p  p|p p|l   Appl -> pple
      stateful=True,
      recurrent_initializer=tf.keras.initializers.GlorotNormal()

  ))

                                                   #embedding layer LSTM   Dense
  model.add(tf.keras.layers.Dense(vocab_size))   #A     |   [1,2,3,1]   |     |   p     
  return model


In [11]:
 #@title show model {form-width: "20%"}
 model=build_model(vocab_size, embedding_dim, rnn_units, batch_size)
 model.summary()
#  tf.keras.utils.plot_model(
#     model,
#     show_shapes=True,
#     show_layer_names=True,
# )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           158976    
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 621)           636525    
Total params: 6,042,477
Trainable params: 6,042,477
Non-trainable params: 0
_________________________________________________________________


In [12]:
#@title loss function & optimizer {form-width: "20%"}
learning_rate = 0.001 #@param {type:"number"}
optimizer = tf.keras.optimizers.Adam(learning_rate)
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer=optimizer, loss=loss)

In [13]:
def get_dir(learning_rate,seq_len,epochs):
    model_id= '_'.join([str(learning_rate),str(seq_len),str(epochs)])
    log_dir=os.path.join('logs',model_id)
    ckpt_dir= os.path.join('train',model_id)
    ckpt_prefix = os.path.join(ckpt_dir, 'ckpt_{epoch}')
    
 
    return log_dir, ckpt_dir, ckpt_prefix 

In [14]:
#@title checkpoints and tarining {form-width: "20%"}
epochs =  150#@param {type:"integer"}


steps_per_epoch=10#@param {type:"number"}
log_dir, ckpt_dir, ckpt_prefix = get_dir(learning_rate,seq_len,epochs)

print(ckpt_dir,ckpt_prefix )

checkpoint_callback=[
                     tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
                     tf.keras.callbacks.ModelCheckpoint(
                         filepath=ckpt_dir,
                         save_weights_only=True,
                        #  mode='min',mointor='val_loss', save_best_only=True,verbose=1
                      ),
                     #tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5),
                     ]

ds=dataset_seqs_batches.repeat()
#model.load_weights((ckpt_dir))

# history = model.fit(
#     x=ds.as_numpy_iterator(),
#     epochs=epochs,
#     steps_per_epoch=steps_per_epoch,
#     callbacks=[checkpoint_callback]  
# )

train/0.001_201_150 train/0.001_201_150/ckpt_{epoch}


# Generate Text

In [15]:
#@title generate text{form-width: "20%"}

def generate_text(model, start_string, num_generate = 1000, temperature=1.0):
    # Evaluation step (generating text using the learned model)

    # Converting our start string to numbers (vectorizing).
    input_indices = [char2index[s] for s in start_string]
    input_indices = tf.expand_dims(input_indices, 0)

    # Empty string to store our results.
    text_generated = []

    # Here batch size == 1.
    model.reset_states()
    for char_index in range(num_generate):
        predictions = model(input_indices)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Using a categorical distribution to predict the character returned by the model.
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(
        predictions,
        num_samples=1
        )[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state.
        input_indices = tf.expand_dims([predicted_id], 0)

        text_generated.append(index2char[predicted_id])

    return (start_string +  ''.join(text_generated))
    

In [16]:
#@title load checkpoints{form-width: "20%"}
simplified_batch_size = 1

pModel=build_model(vocab_size, embedding_dim, rnn_units,batch_size=1)
pModel.load_weights(tf.train.latest_checkpoint(ckpt_dir))
#pModel.load_weights(ckpt_dir)
pModel.build(tf.TensorShape([simplified_batch_size,None]))

print(ckpt_dir,tf.train.latest_checkpoint(ckpt_dir))

# Apple eat by cat
# People afraid of COVID19
# .....

train/0.001_201_150 train/0.001_201_150/ckpt_150


In [17]:
#@title generate wiki{form-width: "20%"}
num_generate=1000 #@param {type:"integer"}
temporature=0.6 #@param {type:"number"}
start_string='Deep learning is '

generated_text=generate_text(pModel,start_string,num_generate,temporature)


import textwrap
for x in textwrap.wrap(generated_text,130,tabsize=0):
  print(x)


Deep learning is a songs in the book to Same became to the current positions in the City of Town of August 1965, and John Wan
International Airportence Comeack. In the reating and Candles (state of the most be sports to Sachonore Englands and 1934.
Category:Townschose People Championship in 1281 south of British and Kensal elective space to his boad after 1960 and the entire
of the night propital of announced in 1973  In ANA However State (Club Was)  2000 (1996)  List of the County French Championship
(1972) (Jert (1986) is a state to the Beit posted by competitions of list in 1987, while the inited in 1350 has been home and
leading the released a north the member of Arence Finall Martinian from North University of Berships in Musquinian and American to
Pendrian Award Afthe County Martinian dames family unere of Spanging simily. The List of Cases and Quiterary 2019.  References
External links Category:American links Category:American program in July 2012 (C  - 3:20 Rusel of American Se

In [19]:
#@title generate wiki{form-width: "20%"}
num_generate=1000 #@param {type:"integer"}
temporature=1.0 #@param {type:"number"}
start_string='Deep learning is '

generated_text=generate_text(pModel,start_string,num_generate,temporature)


import textwrap
for x in textwrap.wrap(generated_text,130,tabsize=0):
  print(x)


Deep learning is with Agenvational pade of Bue Madzezch. Hotil Massachum, he 7 was fiving aghist and allwdays in a vale for bound.
It was from behin 2012 slores of Lock Kanuka, In Goot) As Schose Massilon Aditivist in Effroull Enger monercy and therqume to
villed in diffienchion culture apporated, with see gome Institute Azericand, ascart and proatterage crafing on Jandical
Anterlingte Elmings A Misilia finature to mater and Intluditist Film-70-in Internam daying Sing, International Matherington also
poirf.  The nughnes distributingured to competition and 1/coadde. It was ex stratesition. on 30 Othpart 2iven also Qee Diseu daugn
and the regit totes preticiplid.  West coust severia  Onoughts of Glence Dause Désorationa (Micha Ertrica inlignto M136) was
matteriation of Nuvrall Airporied College Dury 1978  Forewn Depus A="RadKE838 2000) and clinct, a hist and pici of the rine toost
kax also reacing, this who were with a varaction louncis-a grish finary Film 1995.2 18, Stom Such house  of 